In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from lightgbm.sklearn import LGBMRegressor
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
from sklearn.model_selection import GridSearchCV
import datetime
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
import pickle

In [2]:
Train_data = pd.read_csv('./data/used_car_train_20200313.csv', sep=' ')
Test_data = pd.read_csv('./data/used_car_testB_20200421.csv', sep=' ')
# 合并方便后面的操作
df = pd.concat([Train_data, Test_data], ignore_index=True)

In [3]:
df['name_count'] = df.groupby(['name'])['SaleID'].transform('count')
del df['name']
del df['offerType']
del df['seller']

In [4]:
df['price'] = np.log1p(df['price'])

In [5]:
df['fuelType'] = df['fuelType'].fillna(0)
df['gearbox'] = df['gearbox'].fillna(0)
df['bodyType'] = df['bodyType'].fillna(0)
df['model'] = df['model'].fillna(0)

In [6]:
df['power'] = df['power'].map(lambda x: 600 if x>600 else x) #限定power<=600
df['notRepairedDamage'] = df['notRepairedDamage'].astype('str').apply(lambda x: x if x != '-' else None).astype('float32')#类型转换

In [7]:
bin = [i*10 for i in range(31)]
df['power_bin'] = pd.cut(df['power'], bin, labels=False)
bin = [i*10 for i in range(24)]
df['model_bin'] = pd.cut(df['model'], bin, labels=False)

In [8]:
from datetime import datetime
def date_process(x):
    year = int(str(x)[:4])
    month = int(str(x)[4:6])
    day = int(str(x)[6:8])
    if month < 1:
        month = 1
    date = datetime(year, month, day)
    return date
df['regDate'] = df['regDate'].apply(date_process)
df['creatDate'] = df['creatDate'].apply(date_process)
df['regDate_year'] = df['regDate'].dt.year
df['regDate_month'] = df['regDate'].dt.month
df['regDate_day'] = df['regDate'].dt.day
df['creatDate_year'] = df['creatDate'].dt.year
df['creatDate_month'] = df['creatDate'].dt.month
df['creatDate_day'] = df['creatDate'].dt.day
df['car_age_day'] = (df['creatDate'] - df['regDate']).dt.days#二手车使用天数
df['car_age_year'] = round(df['car_age_day'] / 365, 1)#二手车使用年数

In [9]:
cat_cols = ['brand','model','kilometer','fuelType','bodyType']
for col in cat_cols:
    t = Train_data.groupby(col,as_index=False)['price'].agg(
        {col+'_count':'count',col+'_price_max':'max',col+'_price_median':'median',
         col+'_price_min':'min',col+'_price_sum':'sum',col+'_price_std':'std',col+'_price_mean':'mean'})
    df = pd.merge(df,t,on=col,how='left')

In [10]:
kk = ['kilometer','power']
t1 = Train_data.groupby(kk[0],as_index=False)[kk[1]].agg(
        {kk[0]+'_'+kk[1]+'_count':'count',kk[0]+'_'+kk[1]+'_max':'max',kk[0]+'_'+kk[1]+'_median':'median',
         kk[0]+'_'+kk[1]+'_min':'min',kk[0]+'_'+kk[1]+'_sum':'sum',kk[0]+'_'+kk[1]+'_std':'std',kk[0]+'_'+kk[1]+'_mean':'mean'})
df = pd.merge(df,t1,on=kk[0],how='left')

In [11]:
num_cols = [0,3,8,12]
for i in num_cols:
    for j in num_cols:
        df['new'+str(i)+'*'+str(j)]=df['v_'+str(i)]*df['v_'+str(j)]
        
for i in num_cols:
    for j in num_cols:
        df['new'+str(i)+'+'+str(j)]=df['v_'+str(i)]+df['v_'+str(j)]


for i in num_cols:
    for j in num_cols:
        df['new'+str(i)+'-'+str(j)]=df['v_'+str(i)]-df['v_'+str(j)]


for i in range(15):
    df['new'+str(i)+'*year']=df['v_'+str(i)] * df['car_age_year']

In [12]:
df.to_csv('./data/sample_data.gz',compression='gzip', index=0)

In [12]:
print("df information:")
df.info()
df1 = df.copy()
test = df1[df1['price'].isnull()]
X_train = df1[df1['price'].notnull()].drop(['price','regDate','creatDate','SaleID','regionCode'],axis=1)
Y_train = df1[df1['price'].notnull()]['price']
X_test = df1[df1['price'].isnull()].drop(['price','regDate','creatDate','SaleID','regionCode'],axis=1)
print("test information:")
test.info()
print("X_train information:")
X_train.info()

df information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Columns: 144 entries, SaleID to new14*year
dtypes: datetime64[ns](2), float32(1), float64(104), int64(37)
memory usage: 220.5 MB
test information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 150000 to 199999
Columns: 144 entries, SaleID to new14*year
dtypes: datetime64[ns](2), float32(1), float64(104), int64(37)
memory usage: 55.1 MB
X_train information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 150000 entries, 0 to 149999
Columns: 139 entries, model to new14*year
dtypes: float32(1), float64(103), int64(35)
memory usage: 159.6 MB


In [13]:
X_train.columns

Index(['model', 'brand', 'bodyType', 'fuelType', 'gearbox', 'power',
       'kilometer', 'notRepairedDamage', 'v_0', 'v_1',
       ...
       'new5*year', 'new6*year', 'new7*year', 'new8*year', 'new9*year',
       'new10*year', 'new11*year', 'new12*year', 'new13*year', 'new14*year'],
      dtype='object', length=139)

In [14]:
param = {}
param['gpu_id'] = 0
param['tree_method'] = 'gpu_hist'
lgb = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.02,
    boosting_type= 'gbdt',
    objective = 'regression_l1',
    num_leaves=31,
    min_child_samples = 20,
    feature_fraction = 0.8,
    bagging_freq = 1,
    bagging_fraction = 0.8,
    lambda_l2 = 2,
    random_state=2020,
    metric='mae',
    device = 'gpu'
)


param_grid_gb = {
        'depth': [6, 7, 8]
    }
gs_lgb = GridSearchCV(lgb, param_grid_gb)


skf = KFold(n_splits=5)

params_list = []


lgb_mae = 0
sub_lgb = 0
#交叉验证
for i, (trn_idx, val_idx) in enumerate(skf.split(X_train, Y_train)):
    print('--------------------- {} fold ---------------------'.format(i+1))
    trn_x, trn_y = X_train.iloc[trn_idx].reset_index(drop=True), Y_train[trn_idx]
    val_x, val_y = X_train.iloc[val_idx].reset_index(drop=True), Y_train[val_idx]
    # fit函数参数：eval_set=[(x_test,y_test)]  评估数据集,list类型,eval_set用来作为早期停止的验证集，一般我们放x_test和y_test
    # fit函数参数：eval_metric="mlogloss"      评估标准(多分类问题，使用mlogloss作为损失函数)
    # fit函数参数：early_stopping_rounds= 10   如果模型的loss十次内没有减小，则提前结束模型训练
    # fit函数参数：verbose = True              True显示，False不显示
    #import pdb;pdb.set_trace()
    gs_lgb.fit(
        trn_x, trn_y,
        eval_set=[(val_x, val_y)], 
                eval_metric='mae',
        early_stopping_rounds=300,
        #verbose_eval=300
        verbose=False
    )

    params_list.append(gs_lgb.best_params)
    
    sub_lgb += np.expm1(clf.predict(X_test)) / skf.n_splits
    val_lgb = clf.predict(val_x)
    print('val mae:', mean_absolute_error(np.expm1(val_y), np.expm1(val_lgb)))
    lgb_mae += mean_absolute_error(np.expm1(val_y), np.expm1(val_lgb))/skf.n_splits
print('MAE of val with lgb:', lgb_mae)




In [36]:
cat = CatBoostRegressor(loss_function='RMSE',task_type='GPU', 
                        cat_features = ['brand','model','kilometer','fuelType','bodyType'],
                       early_stopping_rounds=300,
                       n_estimators=2000,
                       )

In [37]:
scores = cross_val_score(cat, X_train, Y_train, cv=5, scoring='neg_mean_absolute_error')

In [46]:
# cata_params = {}
# cat_params['cat_features'] = ['brand','model','kilometer','fuelType','bodyType']
    


In [49]:
scores = cross_val_score(cat, X_train, Y_train, cv=5, scoring='neg_mean_absolute_error')

Learning rate set to 0.083382
0:	learn: 1.1288381	total: 20.1ms	remaining: 20.1s
1:	learn: 1.0486638	total: 37.8ms	remaining: 18.9s
2:	learn: 0.9767657	total: 55.5ms	remaining: 18.5s
3:	learn: 0.9101326	total: 74.7ms	remaining: 18.6s
4:	learn: 0.8492627	total: 92.5ms	remaining: 18.4s
5:	learn: 0.7943278	total: 113ms	remaining: 18.7s
6:	learn: 0.7432576	total: 132ms	remaining: 18.7s
7:	learn: 0.6972792	total: 152ms	remaining: 18.8s
8:	learn: 0.6549396	total: 171ms	remaining: 18.8s
9:	learn: 0.6167221	total: 189ms	remaining: 18.7s
10:	learn: 0.5826777	total: 207ms	remaining: 18.6s
11:	learn: 0.5521515	total: 223ms	remaining: 18.4s
12:	learn: 0.5245700	total: 240ms	remaining: 18.2s
13:	learn: 0.4993602	total: 259ms	remaining: 18.3s
14:	learn: 0.4767813	total: 275ms	remaining: 18.1s
15:	learn: 0.4567975	total: 291ms	remaining: 17.9s
16:	learn: 0.4388590	total: 307ms	remaining: 17.8s
17:	learn: 0.4230169	total: 323ms	remaining: 17.6s
18:	learn: 0.4090852	total: 339ms	remaining: 17.5s
19:	le

163:	learn: 0.2476296	total: 2.69s	remaining: 13.7s
164:	learn: 0.2475569	total: 2.7s	remaining: 13.7s
165:	learn: 0.2475183	total: 2.72s	remaining: 13.6s
166:	learn: 0.2474642	total: 2.73s	remaining: 13.6s
167:	learn: 0.2472773	total: 2.75s	remaining: 13.6s
168:	learn: 0.2472438	total: 2.76s	remaining: 13.6s
169:	learn: 0.2471251	total: 2.78s	remaining: 13.6s
170:	learn: 0.2470310	total: 2.79s	remaining: 13.5s
171:	learn: 0.2469359	total: 2.81s	remaining: 13.5s
172:	learn: 0.2468413	total: 2.83s	remaining: 13.5s
173:	learn: 0.2468079	total: 2.85s	remaining: 13.5s
174:	learn: 0.2467797	total: 2.86s	remaining: 13.5s
175:	learn: 0.2467557	total: 2.88s	remaining: 13.5s
176:	learn: 0.2467023	total: 2.9s	remaining: 13.5s
177:	learn: 0.2466017	total: 2.91s	remaining: 13.4s
178:	learn: 0.2465255	total: 2.93s	remaining: 13.4s
179:	learn: 0.2464583	total: 2.94s	remaining: 13.4s
180:	learn: 0.2464221	total: 2.96s	remaining: 13.4s
181:	learn: 0.2463801	total: 2.98s	remaining: 13.4s
182:	learn: 0.

328:	learn: 0.2389310	total: 5.71s	remaining: 11.6s
329:	learn: 0.2389103	total: 5.73s	remaining: 11.6s
330:	learn: 0.2389022	total: 5.74s	remaining: 11.6s
331:	learn: 0.2388894	total: 5.76s	remaining: 11.6s
332:	learn: 0.2388803	total: 5.78s	remaining: 11.6s
333:	learn: 0.2388726	total: 5.79s	remaining: 11.6s
334:	learn: 0.2388657	total: 5.81s	remaining: 11.5s
335:	learn: 0.2388598	total: 5.83s	remaining: 11.5s
336:	learn: 0.2388460	total: 5.85s	remaining: 11.5s
337:	learn: 0.2388413	total: 5.86s	remaining: 11.5s
338:	learn: 0.2388373	total: 5.88s	remaining: 11.5s
339:	learn: 0.2388322	total: 5.9s	remaining: 11.4s
340:	learn: 0.2388127	total: 5.92s	remaining: 11.4s
341:	learn: 0.2387988	total: 5.93s	remaining: 11.4s
342:	learn: 0.2387842	total: 5.95s	remaining: 11.4s
343:	learn: 0.2387708	total: 5.97s	remaining: 11.4s
344:	learn: 0.2387542	total: 5.99s	remaining: 11.4s
345:	learn: 0.2387403	total: 6s	remaining: 11.3s
346:	learn: 0.2387294	total: 6.02s	remaining: 11.3s
347:	learn: 0.23

488:	learn: 0.2360803	total: 8.54s	remaining: 8.93s
489:	learn: 0.2360441	total: 8.56s	remaining: 8.91s
490:	learn: 0.2360329	total: 8.58s	remaining: 8.89s
491:	learn: 0.2360151	total: 8.6s	remaining: 8.88s
492:	learn: 0.2359913	total: 8.61s	remaining: 8.86s
493:	learn: 0.2359558	total: 8.63s	remaining: 8.84s
494:	learn: 0.2359356	total: 8.65s	remaining: 8.83s
495:	learn: 0.2358954	total: 8.67s	remaining: 8.81s
496:	learn: 0.2358819	total: 8.69s	remaining: 8.79s
497:	learn: 0.2358511	total: 8.71s	remaining: 8.78s
498:	learn: 0.2358256	total: 8.73s	remaining: 8.76s
499:	learn: 0.2358142	total: 8.74s	remaining: 8.74s
500:	learn: 0.2358046	total: 8.76s	remaining: 8.73s
501:	learn: 0.2357960	total: 8.79s	remaining: 8.71s
502:	learn: 0.2357888	total: 8.8s	remaining: 8.7s
503:	learn: 0.2357827	total: 8.82s	remaining: 8.68s
504:	learn: 0.2357562	total: 8.84s	remaining: 8.66s
505:	learn: 0.2357511	total: 8.86s	remaining: 8.65s
506:	learn: 0.2357467	total: 8.87s	remaining: 8.63s
507:	learn: 0.2

657:	learn: 0.2335754	total: 11.5s	remaining: 5.97s
658:	learn: 0.2335692	total: 11.5s	remaining: 5.95s
659:	learn: 0.2335638	total: 11.5s	remaining: 5.93s
660:	learn: 0.2335591	total: 11.5s	remaining: 5.92s
661:	learn: 0.2335546	total: 11.6s	remaining: 5.9s
662:	learn: 0.2335508	total: 11.6s	remaining: 5.88s
663:	learn: 0.2335401	total: 11.6s	remaining: 5.86s
664:	learn: 0.2335367	total: 11.6s	remaining: 5.84s
665:	learn: 0.2335339	total: 11.6s	remaining: 5.83s
666:	learn: 0.2335167	total: 11.6s	remaining: 5.81s
667:	learn: 0.2335019	total: 11.7s	remaining: 5.79s
668:	learn: 0.2334994	total: 11.7s	remaining: 5.78s
669:	learn: 0.2334868	total: 11.7s	remaining: 5.76s
670:	learn: 0.2334710	total: 11.7s	remaining: 5.74s
671:	learn: 0.2334672	total: 11.7s	remaining: 5.72s
672:	learn: 0.2334598	total: 11.7s	remaining: 5.71s
673:	learn: 0.2334509	total: 11.8s	remaining: 5.69s
674:	learn: 0.2334225	total: 11.8s	remaining: 5.67s
675:	learn: 0.2334168	total: 11.8s	remaining: 5.65s
676:	learn: 0

...

273:	learn: 0.2450376	total: 5.04s	remaining: 13.4s
274:	learn: 0.2450357	total: 5.06s	remaining: 13.3s
275:	learn: 0.2450097	total: 5.08s	remaining: 13.3s
276:	learn: 0.2449581	total: 5.1s	remaining: 13.3s
277:	learn: 0.2449224	total: 5.12s	remaining: 13.3s
278:	learn: 0.2448786	total: 5.14s	remaining: 13.3s
279:	learn: 0.2448767	total: 5.16s	remaining: 13.3s
280:	learn: 0.2448393	total: 5.18s	remaining: 13.2s
281:	learn: 0.2448088	total: 5.2s	remaining: 13.2s
282:	learn: 0.2448070	total: 5.21s	remaining: 13.2s
283:	learn: 0.2448036	total: 5.23s	remaining: 13.2s
284:	learn: 0.2447832	total: 5.25s	remaining: 13.2s
285:	learn: 0.2447658	total: 5.27s	remaining: 13.2s
286:	learn: 0.2447335	total: 5.29s	remaining: 13.2s
287:	learn: 0.2447185	total: 5.31s	remaining: 13.1s
288:	learn: 0.2447056	total: 5.33s	remaining: 13.1s
289:	learn: 0.2446778	total: 5.35s	remaining: 13.1s
290:	learn: 0.2446542	total: 5.37s	remaining: 13.1s
291:	learn: 0.2446278	total: 5.39s	remaining: 13.1s
292:	learn: 0.

435:	learn: 0.2438462	total: 7.91s	remaining: 10.2s
436:	learn: 0.2438462	total: 7.93s	remaining: 10.2s
437:	learn: 0.2438462	total: 7.95s	remaining: 10.2s
438:	learn: 0.2438462	total: 7.96s	remaining: 10.2s
439:	learn: 0.2438462	total: 7.98s	remaining: 10.2s
440:	learn: 0.2438462	total: 8s	remaining: 10.1s
441:	learn: 0.2438461	total: 8.02s	remaining: 10.1s
442:	learn: 0.2438461	total: 8.04s	remaining: 10.1s
443:	learn: 0.2438461	total: 8.05s	remaining: 10.1s
444:	learn: 0.2438461	total: 8.07s	remaining: 10.1s
445:	learn: 0.2438461	total: 8.09s	remaining: 10s
446:	learn: 0.2438461	total: 8.1s	remaining: 10s
447:	learn: 0.2438461	total: 8.12s	remaining: 10s
448:	learn: 0.2438461	total: 8.14s	remaining: 9.99s
449:	learn: 0.2438461	total: 8.16s	remaining: 9.97s
450:	learn: 0.2438461	total: 8.18s	remaining: 9.95s
451:	learn: 0.2438461	total: 8.19s	remaining: 9.94s
452:	learn: 0.2438461	total: 8.21s	remaining: 9.92s
453:	learn: 0.2438461	total: 8.23s	remaining: 9.9s
454:	learn: 0.2438460	t

603:	learn: 0.2438460	total: 10.9s	remaining: 7.13s
604:	learn: 0.2438460	total: 10.9s	remaining: 7.11s
605:	learn: 0.2438460	total: 10.9s	remaining: 7.09s
606:	learn: 0.2438460	total: 10.9s	remaining: 7.08s
607:	learn: 0.2438460	total: 10.9s	remaining: 7.06s
608:	learn: 0.2438460	total: 11s	remaining: 7.04s
609:	learn: 0.2438460	total: 11s	remaining: 7.02s
610:	learn: 0.2438460	total: 11s	remaining: 7s
611:	learn: 0.2438460	total: 11s	remaining: 6.98s
612:	learn: 0.2438460	total: 11s	remaining: 6.96s
613:	learn: 0.2438460	total: 11s	remaining: 6.95s
614:	learn: 0.2438460	total: 11.1s	remaining: 6.93s
615:	learn: 0.2438460	total: 11.1s	remaining: 6.91s
616:	learn: 0.2438460	total: 11.1s	remaining: 6.89s
617:	learn: 0.2438460	total: 11.1s	remaining: 6.87s
618:	learn: 0.2438460	total: 11.1s	remaining: 6.85s
619:	learn: 0.2438460	total: 11.2s	remaining: 6.84s
620:	learn: 0.2438460	total: 11.2s	remaining: 6.82s
621:	learn: 0.2438460	total: 11.2s	remaining: 6.8s
622:	learn: 0.2438460	total:

765:	learn: 0.2438460	total: 13.7s	remaining: 4.19s
766:	learn: 0.2438460	total: 13.7s	remaining: 4.18s
767:	learn: 0.2438460	total: 13.8s	remaining: 4.16s
768:	learn: 0.2438460	total: 13.8s	remaining: 4.14s
769:	learn: 0.2438460	total: 13.8s	remaining: 4.12s
770:	learn: 0.2438460	total: 13.8s	remaining: 4.1s
771:	learn: 0.2438460	total: 13.8s	remaining: 4.09s
772:	learn: 0.2438460	total: 13.9s	remaining: 4.07s
773:	learn: 0.2438460	total: 13.9s	remaining: 4.05s
774:	learn: 0.2438460	total: 13.9s	remaining: 4.03s
775:	learn: 0.2438460	total: 13.9s	remaining: 4.01s
776:	learn: 0.2438460	total: 13.9s	remaining: 4s
777:	learn: 0.2438460	total: 13.9s	remaining: 3.98s
778:	learn: 0.2438460	total: 14s	remaining: 3.96s
779:	learn: 0.2438460	total: 14s	remaining: 3.94s
780:	learn: 0.2438460	total: 14s	remaining: 3.92s
781:	learn: 0.2438460	total: 14s	remaining: 3.9s
782:	learn: 0.2438460	total: 14s	remaining: 3.89s
783:	learn: 0.2438460	total: 14s	remaining: 3.87s
784:	learn: 0.2438460	total: 

931:	learn: 0.2438460	total: 16.7s	remaining: 1.22s
932:	learn: 0.2438460	total: 16.7s	remaining: 1.2s
933:	learn: 0.2438460	total: 16.7s	remaining: 1.18s
934:	learn: 0.2438460	total: 16.8s	remaining: 1.17s
935:	learn: 0.2438460	total: 16.8s	remaining: 1.15s
936:	learn: 0.2438460	total: 16.8s	remaining: 1.13s
937:	learn: 0.2438460	total: 16.8s	remaining: 1.11s
938:	learn: 0.2438460	total: 16.8s	remaining: 1.09s
939:	learn: 0.2438460	total: 16.9s	remaining: 1.07s
940:	learn: 0.2438460	total: 16.9s	remaining: 1.06s
941:	learn: 0.2438460	total: 16.9s	remaining: 1.04s
942:	learn: 0.2438460	total: 16.9s	remaining: 1.02s
943:	learn: 0.2438460	total: 16.9s	remaining: 1s
944:	learn: 0.2438460	total: 16.9s	remaining: 986ms
945:	learn: 0.2438460	total: 17s	remaining: 968ms
946:	learn: 0.2438460	total: 17s	remaining: 950ms
947:	learn: 0.2438460	total: 17s	remaining: 932ms
948:	learn: 0.2438460	total: 17s	remaining: 914ms
949:	learn: 0.2438460	total: 17s	remaining: 896ms
950:	learn: 0.2438460	tota

In [17]:
print(scores*-3000)

[nan nan nan nan nan]


In [57]:
cat_mae = np.sum(scores*-3000)/5

In [51]:
cat.fit(X_train, Y_train)

Learning rate set to 0.084261
0:	learn: 1.1285656	total: 23.4ms	remaining: 23.4s
1:	learn: 1.0471957	total: 44.1ms	remaining: 22s
2:	learn: 0.9730130	total: 64.9ms	remaining: 21.6s
3:	learn: 0.9055936	total: 86.2ms	remaining: 21.5s
4:	learn: 0.8441886	total: 106ms	remaining: 21.2s
5:	learn: 0.7885177	total: 125ms	remaining: 20.6s
6:	learn: 0.7381219	total: 145ms	remaining: 20.5s
7:	learn: 0.6923455	total: 164ms	remaining: 20.3s
8:	learn: 0.6500307	total: 184ms	remaining: 20.3s
9:	learn: 0.6120824	total: 204ms	remaining: 20.2s
10:	learn: 0.5784972	total: 223ms	remaining: 20s
11:	learn: 0.5485582	total: 243ms	remaining: 20s
12:	learn: 0.5208343	total: 261ms	remaining: 19.9s
13:	learn: 0.4958030	total: 282ms	remaining: 19.9s
14:	learn: 0.4742371	total: 303ms	remaining: 19.9s
15:	learn: 0.4541292	total: 323ms	remaining: 19.9s
16:	learn: 0.4362352	total: 344ms	remaining: 19.9s
17:	learn: 0.4199907	total: 364ms	remaining: 19.8s
18:	learn: 0.4061499	total: 387ms	remaining: 20s
19:	learn: 0.39

165:	learn: 0.2432779	total: 3.36s	remaining: 16.9s
166:	learn: 0.2431895	total: 3.38s	remaining: 16.9s
167:	learn: 0.2431277	total: 3.4s	remaining: 16.9s
168:	learn: 0.2430837	total: 3.42s	remaining: 16.8s
169:	learn: 0.2430440	total: 3.44s	remaining: 16.8s
170:	learn: 0.2429990	total: 3.46s	remaining: 16.8s
171:	learn: 0.2429609	total: 3.48s	remaining: 16.7s
172:	learn: 0.2429414	total: 3.49s	remaining: 16.7s
173:	learn: 0.2428904	total: 3.51s	remaining: 16.7s
174:	learn: 0.2428739	total: 3.53s	remaining: 16.6s
175:	learn: 0.2428598	total: 3.55s	remaining: 16.6s
176:	learn: 0.2428212	total: 3.57s	remaining: 16.6s
177:	learn: 0.2428093	total: 3.59s	remaining: 16.6s
178:	learn: 0.2427700	total: 3.61s	remaining: 16.5s
179:	learn: 0.2426265	total: 3.63s	remaining: 16.5s
180:	learn: 0.2426163	total: 3.65s	remaining: 16.5s
181:	learn: 0.2425933	total: 3.67s	remaining: 16.5s
182:	learn: 0.2425554	total: 3.68s	remaining: 16.4s
183:	learn: 0.2425234	total: 3.7s	remaining: 16.4s
184:	learn: 0.

327:	learn: 0.2371717	total: 6.63s	remaining: 13.6s
328:	learn: 0.2371656	total: 6.65s	remaining: 13.6s
329:	learn: 0.2371611	total: 6.67s	remaining: 13.5s
330:	learn: 0.2371558	total: 6.7s	remaining: 13.5s
331:	learn: 0.2371479	total: 6.72s	remaining: 13.5s
332:	learn: 0.2371311	total: 6.74s	remaining: 13.5s
333:	learn: 0.2371077	total: 6.75s	remaining: 13.5s
334:	learn: 0.2370879	total: 6.77s	remaining: 13.4s
335:	learn: 0.2370711	total: 6.79s	remaining: 13.4s
336:	learn: 0.2370337	total: 6.81s	remaining: 13.4s
337:	learn: 0.2370195	total: 6.83s	remaining: 13.4s
338:	learn: 0.2370036	total: 6.84s	remaining: 13.3s
339:	learn: 0.2369882	total: 6.86s	remaining: 13.3s
340:	learn: 0.2368990	total: 6.88s	remaining: 13.3s
341:	learn: 0.2368953	total: 6.91s	remaining: 13.3s
342:	learn: 0.2368794	total: 6.93s	remaining: 13.3s
343:	learn: 0.2368763	total: 6.95s	remaining: 13.3s
344:	learn: 0.2368654	total: 6.97s	remaining: 13.2s
345:	learn: 0.2368626	total: 6.99s	remaining: 13.2s
346:	learn: 0

490:	learn: 0.2364930	total: 10.1s	remaining: 10.5s
491:	learn: 0.2364930	total: 10.1s	remaining: 10.4s
492:	learn: 0.2364930	total: 10.1s	remaining: 10.4s
493:	learn: 0.2364930	total: 10.2s	remaining: 10.4s
494:	learn: 0.2364930	total: 10.2s	remaining: 10.4s
495:	learn: 0.2364930	total: 10.2s	remaining: 10.4s
496:	learn: 0.2364929	total: 10.2s	remaining: 10.3s
497:	learn: 0.2364930	total: 10.2s	remaining: 10.3s
498:	learn: 0.2364929	total: 10.3s	remaining: 10.3s
499:	learn: 0.2364929	total: 10.3s	remaining: 10.3s
500:	learn: 0.2364929	total: 10.3s	remaining: 10.3s
501:	learn: 0.2364929	total: 10.3s	remaining: 10.2s
502:	learn: 0.2364929	total: 10.3s	remaining: 10.2s
503:	learn: 0.2364929	total: 10.4s	remaining: 10.2s
504:	learn: 0.2364929	total: 10.4s	remaining: 10.2s
505:	learn: 0.2364929	total: 10.4s	remaining: 10.2s
506:	learn: 0.2364929	total: 10.4s	remaining: 10.1s
507:	learn: 0.2364929	total: 10.4s	remaining: 10.1s
508:	learn: 0.2364929	total: 10.5s	remaining: 10.1s
509:	learn: 

650:	learn: 0.2364929	total: 13.5s	remaining: 7.22s
651:	learn: 0.2364929	total: 13.5s	remaining: 7.2s
652:	learn: 0.2364929	total: 13.5s	remaining: 7.18s
653:	learn: 0.2364929	total: 13.5s	remaining: 7.16s
654:	learn: 0.2364929	total: 13.6s	remaining: 7.14s
655:	learn: 0.2364929	total: 13.6s	remaining: 7.12s
656:	learn: 0.2364929	total: 13.6s	remaining: 7.1s
657:	learn: 0.2364929	total: 13.6s	remaining: 7.08s
658:	learn: 0.2364929	total: 13.6s	remaining: 7.06s
659:	learn: 0.2364929	total: 13.7s	remaining: 7.04s
660:	learn: 0.2364929	total: 13.7s	remaining: 7.02s
661:	learn: 0.2364929	total: 13.7s	remaining: 7s
662:	learn: 0.2364929	total: 13.7s	remaining: 6.98s
663:	learn: 0.2364929	total: 13.7s	remaining: 6.96s
664:	learn: 0.2364929	total: 13.8s	remaining: 6.94s
665:	learn: 0.2364929	total: 13.8s	remaining: 6.92s
666:	learn: 0.2364929	total: 13.8s	remaining: 6.9s
667:	learn: 0.2364929	total: 13.8s	remaining: 6.88s
668:	learn: 0.2364929	total: 13.9s	remaining: 6.86s
669:	learn: 0.2364

810:	learn: 0.2364929	total: 16.9s	remaining: 3.94s
811:	learn: 0.2364929	total: 16.9s	remaining: 3.91s
812:	learn: 0.2364929	total: 16.9s	remaining: 3.89s
813:	learn: 0.2364929	total: 17s	remaining: 3.87s
814:	learn: 0.2364929	total: 17s	remaining: 3.85s
815:	learn: 0.2364929	total: 17s	remaining: 3.83s
816:	learn: 0.2364929	total: 17s	remaining: 3.81s
817:	learn: 0.2364929	total: 17s	remaining: 3.79s
818:	learn: 0.2364929	total: 17.1s	remaining: 3.77s
819:	learn: 0.2364929	total: 17.1s	remaining: 3.75s
820:	learn: 0.2364929	total: 17.1s	remaining: 3.73s
821:	learn: 0.2364929	total: 17.1s	remaining: 3.71s
822:	learn: 0.2364929	total: 17.1s	remaining: 3.69s
823:	learn: 0.2364929	total: 17.2s	remaining: 3.66s
824:	learn: 0.2364929	total: 17.2s	remaining: 3.64s
825:	learn: 0.2364929	total: 17.2s	remaining: 3.62s
826:	learn: 0.2364929	total: 17.2s	remaining: 3.6s
827:	learn: 0.2364929	total: 17.2s	remaining: 3.58s
828:	learn: 0.2364929	total: 17.3s	remaining: 3.56s
829:	learn: 0.2364929	t

971:	learn: 0.2364929	total: 20.3s	remaining: 586ms
972:	learn: 0.2364929	total: 20.4s	remaining: 565ms
973:	learn: 0.2364929	total: 20.4s	remaining: 544ms
974:	learn: 0.2364929	total: 20.4s	remaining: 523ms
975:	learn: 0.2364929	total: 20.4s	remaining: 502ms
976:	learn: 0.2364929	total: 20.4s	remaining: 481ms
977:	learn: 0.2364929	total: 20.5s	remaining: 460ms
978:	learn: 0.2364929	total: 20.5s	remaining: 439ms
979:	learn: 0.2364929	total: 20.5s	remaining: 418ms
980:	learn: 0.2364929	total: 20.5s	remaining: 398ms
981:	learn: 0.2364929	total: 20.5s	remaining: 377ms
982:	learn: 0.2364929	total: 20.6s	remaining: 356ms
983:	learn: 0.2364929	total: 20.6s	remaining: 335ms
984:	learn: 0.2364929	total: 20.6s	remaining: 314ms
985:	learn: 0.2364929	total: 20.6s	remaining: 293ms
986:	learn: 0.2364929	total: 20.7s	remaining: 272ms
987:	learn: 0.2364929	total: 20.7s	remaining: 251ms
988:	learn: 0.2364929	total: 20.7s	remaining: 230ms
989:	learn: 0.2364929	total: 20.7s	remaining: 209ms
990:	learn: 

In [67]:
cat_sub = np.expm1(cat.predict(X_test))

In [68]:
print(cat_sub)

[1374.3072223  2043.04565629 8211.50870485 ... 5846.98875832 4842.50304292
 4645.03884163]


In [53]:
sub_lgb

array([1269.95937553, 1979.92968263, 8410.87197818, ..., 5502.34935444,
       4796.22764772, 5164.62750762])

In [40]:
# 保存模型！！！！！！！！！！！！！！！

In [45]:
lgb_mae

459.9862144686087

In [55]:
save_file = open('lgb.pkl', 'wb')
# 将列表写入文件
pickle.dump(lgb_mae, save_file)
pickle.dump(sub_lgb, save_file)
save_file.close()

In [40]:
pickle_file = open('lgb.pkl','rb')
lgb_mae = pickle.load(pickle_file)
sub_lgb = pickle.load(pickle_file)

In [41]:
lgb_mae

459.9862144686087

In [62]:
sub_lgb

array([1269.95937553, 1979.92968263, 8410.87197818, ..., 5502.34935444,
       4796.22764772, 5164.62750762])

In [61]:
sub_Weighted = (1-lgb_mae/(cat_mae+lgb_mae))*sub_lgb+(1-cat_mae/(cat_mae+lgb_mae))*sub_cat

In [62]:
sub_Weighted

array([1284.52914321, 1928.65695293, 8353.30617466, ..., 5529.21437826,
       4916.39433335, 5078.34418537])

In [63]:
res = {}
res['SaleID'] = Test_data['SaleID']
res['price'] = sub_Weighted

res_df = pd.DataFrame(res)

res_df.to_csv('./data/cat_lgb_gs.csv', index=None)

In [38]:
skf = KFold(n_splits=5)
cat = CatBoostRegressor(loss_function='RMSE',task_type='GPU', 
#                         cat_features = ['brand','model','kilometer','fuelType','bodyType'],
                       early_stopping_rounds=300,
                       )
param_grid = {
        'depth': [6, 7, 8]
    }
gs = GridSearchCV(cat, param_grid)
cat_mae = 0
sub_cat = 0
for i, (trn_idx, val_idx) in enumerate(skf.split(X_train, Y_train)):
    print('--------------------- {} fold ---------------------'.format(i+1))
    print(datetime.now())
    trn_x, trn_y = X_train.iloc[trn_idx].reset_index(drop=True), Y_train[trn_idx]
    val_x, val_y = X_train.iloc[val_idx].reset_index(drop=True), Y_train[val_idx]
    # fit函数参数：eval_set=[(x_test,y_test)]  评估数据集,list类型,eval_set用来作为早期停止的验证集，一般我们放x_test和y_test
    # fit函数参数：eval_metric="mlogloss"      评估标准(多分类问题，使用mlogloss作为损失函数)
    # fit函数参数：early_stopping_rounds= 10   如果模型的loss十次内没有减小，则提前结束模型训练
    # fit函数参数：verbose = True              True显示，False不显示
    gs.fit(
        trn_x, trn_y,
        verbose=False
    )
    sub_cat += np.expm1(gs.predict(X_test)) / skf.n_splits
    val_cat= gs.predict(val_x)
    print('val mae:', mean_absolute_error(np.expm1(val_y), np.expm1(val_cat)))
    cat_mae += mean_absolute_error(np.expm1(val_y), np.expm1(val_cat))/skf.n_splits
    print(datetime.now())

print('MAE of val with xgb:', cat_mae)

--------------------- 1 fold ---------------------
2020-08-26 22:49:58.320843
val mae: 672.6561060439875
2020-08-26 22:56:36.744313
--------------------- 2 fold ---------------------
2020-08-26 22:56:36.745311
val mae: 658.3483623935076
2020-08-26 23:03:18.791313
--------------------- 3 fold ---------------------
2020-08-26 23:03:18.792311
val mae: 679.9992479520395
2020-08-26 23:10:01.025194
--------------------- 4 fold ---------------------
2020-08-26 23:10:01.026192
val mae: 663.2956890046644
2020-08-26 23:16:36.448688
--------------------- 5 fold ---------------------
2020-08-26 23:16:36.449653
val mae: 666.6724620580853
2020-08-26 23:23:15.360106
MAE of val with xgb: 668.1943734904569


In [57]:
cat = CatBoostRegressor(loss_function='RMSE',task_type='GPU', 
#                         cat_features = ['brand','model','kilometer','fuelType','bodyType'],
                       early_stopping_rounds=500,
                        iterations = 5000,
                    depth=8)

In [58]:
cat.fit(X_train, Y_train)

Learning rate set to 0.032549
0:	learn: 1.1831322	total: 34.7ms	remaining: 2m 53s
1:	learn: 1.1489788	total: 70.7ms	remaining: 2m 56s
2:	learn: 1.1162290	total: 101ms	remaining: 2m 48s
3:	learn: 1.0846083	total: 134ms	remaining: 2m 46s
4:	learn: 1.0540949	total: 166ms	remaining: 2m 45s
5:	learn: 1.0247495	total: 196ms	remaining: 2m 43s
6:	learn: 0.9957882	total: 229ms	remaining: 2m 43s
7:	learn: 0.9679686	total: 263ms	remaining: 2m 44s
8:	learn: 0.9415998	total: 293ms	remaining: 2m 42s
9:	learn: 0.9159255	total: 324ms	remaining: 2m 41s
10:	learn: 0.8913803	total: 353ms	remaining: 2m 40s
11:	learn: 0.8674826	total: 385ms	remaining: 2m 40s
12:	learn: 0.8443901	total: 415ms	remaining: 2m 39s
13:	learn: 0.8221020	total: 446ms	remaining: 2m 38s
14:	learn: 0.8007479	total: 479ms	remaining: 2m 39s
15:	learn: 0.7802168	total: 512ms	remaining: 2m 39s
16:	learn: 0.7600073	total: 545ms	remaining: 2m 39s
17:	learn: 0.7406981	total: 578ms	remaining: 2m 39s
18:	learn: 0.7219347	total: 612ms	remainin

162:	learn: 0.2510266	total: 5.32s	remaining: 2m 37s
163:	learn: 0.2508613	total: 5.35s	remaining: 2m 37s
164:	learn: 0.2506857	total: 5.38s	remaining: 2m 37s
165:	learn: 0.2505218	total: 5.42s	remaining: 2m 37s
166:	learn: 0.2503662	total: 5.45s	remaining: 2m 37s
167:	learn: 0.2502163	total: 5.48s	remaining: 2m 37s
168:	learn: 0.2500401	total: 5.51s	remaining: 2m 37s
169:	learn: 0.2499183	total: 5.54s	remaining: 2m 37s
170:	learn: 0.2496927	total: 5.57s	remaining: 2m 37s
171:	learn: 0.2495450	total: 5.6s	remaining: 2m 37s
172:	learn: 0.2493972	total: 5.63s	remaining: 2m 37s
173:	learn: 0.2491409	total: 5.66s	remaining: 2m 37s
174:	learn: 0.2489831	total: 5.7s	remaining: 2m 37s
175:	learn: 0.2488551	total: 5.72s	remaining: 2m 36s
176:	learn: 0.2487686	total: 5.75s	remaining: 2m 36s
177:	learn: 0.2486281	total: 5.78s	remaining: 2m 36s
178:	learn: 0.2484539	total: 5.81s	remaining: 2m 36s
179:	learn: 0.2482580	total: 5.84s	remaining: 2m 36s
180:	learn: 0.2481245	total: 5.88s	remaining: 2m

321:	learn: 0.2360441	total: 10.2s	remaining: 2m 28s
322:	learn: 0.2359997	total: 10.3s	remaining: 2m 28s
323:	learn: 0.2359330	total: 10.3s	remaining: 2m 28s
324:	learn: 0.2358982	total: 10.3s	remaining: 2m 28s
325:	learn: 0.2358582	total: 10.3s	remaining: 2m 28s
326:	learn: 0.2357945	total: 10.4s	remaining: 2m 28s
327:	learn: 0.2357663	total: 10.4s	remaining: 2m 28s
328:	learn: 0.2357252	total: 10.4s	remaining: 2m 28s
329:	learn: 0.2356816	total: 10.5s	remaining: 2m 28s
330:	learn: 0.2356503	total: 10.5s	remaining: 2m 28s
331:	learn: 0.2355647	total: 10.5s	remaining: 2m 28s
332:	learn: 0.2355115	total: 10.6s	remaining: 2m 28s
333:	learn: 0.2354587	total: 10.6s	remaining: 2m 28s
334:	learn: 0.2354147	total: 10.6s	remaining: 2m 28s
335:	learn: 0.2353634	total: 10.7s	remaining: 2m 27s
336:	learn: 0.2352327	total: 10.7s	remaining: 2m 27s
337:	learn: 0.2351712	total: 10.7s	remaining: 2m 27s
338:	learn: 0.2351216	total: 10.8s	remaining: 2m 27s
339:	learn: 0.2350863	total: 10.8s	remaining: 

479:	learn: 0.2299290	total: 15s	remaining: 2m 21s
480:	learn: 0.2298988	total: 15s	remaining: 2m 21s
481:	learn: 0.2298561	total: 15.1s	remaining: 2m 21s
482:	learn: 0.2298415	total: 15.1s	remaining: 2m 21s
483:	learn: 0.2298088	total: 15.1s	remaining: 2m 21s
484:	learn: 0.2297852	total: 15.2s	remaining: 2m 21s
485:	learn: 0.2297533	total: 15.2s	remaining: 2m 21s
486:	learn: 0.2297295	total: 15.2s	remaining: 2m 21s
487:	learn: 0.2297071	total: 15.3s	remaining: 2m 21s
488:	learn: 0.2296848	total: 15.3s	remaining: 2m 20s
489:	learn: 0.2296580	total: 15.3s	remaining: 2m 20s
490:	learn: 0.2296206	total: 15.3s	remaining: 2m 20s
491:	learn: 0.2295926	total: 15.4s	remaining: 2m 20s
492:	learn: 0.2295724	total: 15.4s	remaining: 2m 20s
493:	learn: 0.2295425	total: 15.4s	remaining: 2m 20s
494:	learn: 0.2295142	total: 15.5s	remaining: 2m 20s
495:	learn: 0.2294758	total: 15.5s	remaining: 2m 20s
496:	learn: 0.2294501	total: 15.5s	remaining: 2m 20s
497:	learn: 0.2294129	total: 15.5s	remaining: 2m 2

636:	learn: 0.2265582	total: 19.8s	remaining: 2m 15s
637:	learn: 0.2265375	total: 19.8s	remaining: 2m 15s
638:	learn: 0.2265142	total: 19.8s	remaining: 2m 15s
639:	learn: 0.2264921	total: 19.9s	remaining: 2m 15s
640:	learn: 0.2264782	total: 19.9s	remaining: 2m 15s
641:	learn: 0.2264636	total: 19.9s	remaining: 2m 15s
642:	learn: 0.2264358	total: 20s	remaining: 2m 15s
643:	learn: 0.2264214	total: 20s	remaining: 2m 15s
644:	learn: 0.2264082	total: 20s	remaining: 2m 15s
645:	learn: 0.2263677	total: 20.1s	remaining: 2m 15s
646:	learn: 0.2263475	total: 20.1s	remaining: 2m 15s
647:	learn: 0.2263285	total: 20.1s	remaining: 2m 15s
648:	learn: 0.2263079	total: 20.1s	remaining: 2m 15s
649:	learn: 0.2262742	total: 20.2s	remaining: 2m 15s
650:	learn: 0.2262564	total: 20.2s	remaining: 2m 14s
651:	learn: 0.2262444	total: 20.2s	remaining: 2m 14s
652:	learn: 0.2262284	total: 20.3s	remaining: 2m 14s
653:	learn: 0.2262132	total: 20.3s	remaining: 2m 14s
654:	learn: 0.2261989	total: 20.3s	remaining: 2m 14s

...

4232:	learn: 0.1997575	total: 2m 18s	remaining: 25.1s
4233:	learn: 0.1997575	total: 2m 18s	remaining: 25s
4234:	learn: 0.1997575	total: 2m 18s	remaining: 25s
4235:	learn: 0.1997575	total: 2m 18s	remaining: 25s
4236:	learn: 0.1997575	total: 2m 18s	remaining: 24.9s
4237:	learn: 0.1997575	total: 2m 18s	remaining: 24.9s
4238:	learn: 0.1997575	total: 2m 18s	remaining: 24.9s
4239:	learn: 0.1997575	total: 2m 18s	remaining: 24.8s
4240:	learn: 0.1997575	total: 2m 18s	remaining: 24.8s
4241:	learn: 0.1997575	total: 2m 18s	remaining: 24.8s
4242:	learn: 0.1997575	total: 2m 18s	remaining: 24.7s
4243:	learn: 0.1997575	total: 2m 18s	remaining: 24.7s
4244:	learn: 0.1997575	total: 2m 18s	remaining: 24.7s
4245:	learn: 0.1997575	total: 2m 18s	remaining: 24.6s
4246:	learn: 0.1997575	total: 2m 18s	remaining: 24.6s
4247:	learn: 0.1997575	total: 2m 18s	remaining: 24.6s
4248:	learn: 0.1997575	total: 2m 18s	remaining: 24.5s
4249:	learn: 0.1997575	total: 2m 18s	remaining: 24.5s
4250:	learn: 0.1997575	total: 2m 1

4385:	learn: 0.1997574	total: 2m 23s	remaining: 20s
4386:	learn: 0.1997574	total: 2m 23s	remaining: 20s
4387:	learn: 0.1997574	total: 2m 23s	remaining: 20s
4388:	learn: 0.1997574	total: 2m 23s	remaining: 19.9s
4389:	learn: 0.1997574	total: 2m 23s	remaining: 19.9s
4390:	learn: 0.1997574	total: 2m 23s	remaining: 19.9s
4391:	learn: 0.1997574	total: 2m 23s	remaining: 19.8s
4392:	learn: 0.1997574	total: 2m 23s	remaining: 19.8s
4393:	learn: 0.1997574	total: 2m 23s	remaining: 19.8s
4394:	learn: 0.1997574	total: 2m 23s	remaining: 19.7s
4395:	learn: 0.1997574	total: 2m 23s	remaining: 19.7s
4396:	learn: 0.1997574	total: 2m 23s	remaining: 19.7s
4397:	learn: 0.1997574	total: 2m 23s	remaining: 19.6s
4398:	learn: 0.1997574	total: 2m 23s	remaining: 19.6s
4399:	learn: 0.1997574	total: 2m 23s	remaining: 19.6s
4400:	learn: 0.1997574	total: 2m 23s	remaining: 19.5s
4401:	learn: 0.1997574	total: 2m 23s	remaining: 19.5s
4402:	learn: 0.1997574	total: 2m 23s	remaining: 19.5s
4403:	learn: 0.1997574	total: 2m 2

4541:	learn: 0.1997574	total: 2m 27s	remaining: 14.9s
4542:	learn: 0.1997574	total: 2m 27s	remaining: 14.9s
4543:	learn: 0.1997574	total: 2m 27s	remaining: 14.8s
4544:	learn: 0.1997574	total: 2m 27s	remaining: 14.8s
4545:	learn: 0.1997574	total: 2m 28s	remaining: 14.8s
4546:	learn: 0.1997574	total: 2m 28s	remaining: 14.7s
4547:	learn: 0.1997574	total: 2m 28s	remaining: 14.7s
4548:	learn: 0.1997574	total: 2m 28s	remaining: 14.7s
4549:	learn: 0.1997574	total: 2m 28s	remaining: 14.7s
4550:	learn: 0.1997574	total: 2m 28s	remaining: 14.6s
4551:	learn: 0.1997574	total: 2m 28s	remaining: 14.6s
4552:	learn: 0.1997574	total: 2m 28s	remaining: 14.6s
4553:	learn: 0.1997574	total: 2m 28s	remaining: 14.5s
4554:	learn: 0.1997574	total: 2m 28s	remaining: 14.5s
4555:	learn: 0.1997574	total: 2m 28s	remaining: 14.5s
4556:	learn: 0.1997574	total: 2m 28s	remaining: 14.4s
4557:	learn: 0.1997574	total: 2m 28s	remaining: 14.4s
4558:	learn: 0.1997574	total: 2m 28s	remaining: 14.4s
4559:	learn: 0.1997574	total

4695:	learn: 0.1997574	total: 2m 32s	remaining: 9.88s
4696:	learn: 0.1997574	total: 2m 32s	remaining: 9.85s
4697:	learn: 0.1997574	total: 2m 32s	remaining: 9.81s
4698:	learn: 0.1997574	total: 2m 32s	remaining: 9.78s
4699:	learn: 0.1997574	total: 2m 32s	remaining: 9.75s
4700:	learn: 0.1997574	total: 2m 32s	remaining: 9.72s
4701:	learn: 0.1997574	total: 2m 32s	remaining: 9.68s
4702:	learn: 0.1997574	total: 2m 32s	remaining: 9.65s
4703:	learn: 0.1997574	total: 2m 32s	remaining: 9.62s
4704:	learn: 0.1997574	total: 2m 32s	remaining: 9.59s
4705:	learn: 0.1997574	total: 2m 32s	remaining: 9.55s
4706:	learn: 0.1997574	total: 2m 32s	remaining: 9.52s
4707:	learn: 0.1997574	total: 2m 32s	remaining: 9.49s
4708:	learn: 0.1997574	total: 2m 33s	remaining: 9.46s
4709:	learn: 0.1997574	total: 2m 33s	remaining: 9.42s
4710:	learn: 0.1997574	total: 2m 33s	remaining: 9.39s
4711:	learn: 0.1997574	total: 2m 33s	remaining: 9.36s
4712:	learn: 0.1997574	total: 2m 33s	remaining: 9.32s
4713:	learn: 0.1997574	total

4851:	learn: 0.1997574	total: 2m 37s	remaining: 4.8s
4852:	learn: 0.1997574	total: 2m 37s	remaining: 4.77s
4853:	learn: 0.1997574	total: 2m 37s	remaining: 4.74s
4854:	learn: 0.1997574	total: 2m 37s	remaining: 4.7s
4855:	learn: 0.1997574	total: 2m 37s	remaining: 4.67s
4856:	learn: 0.1997574	total: 2m 37s	remaining: 4.64s
4857:	learn: 0.1997574	total: 2m 37s	remaining: 4.61s
4858:	learn: 0.1997574	total: 2m 37s	remaining: 4.57s
4859:	learn: 0.1997574	total: 2m 37s	remaining: 4.54s
4860:	learn: 0.1997574	total: 2m 37s	remaining: 4.51s
4861:	learn: 0.1997574	total: 2m 37s	remaining: 4.47s
4862:	learn: 0.1997574	total: 2m 37s	remaining: 4.44s
4863:	learn: 0.1997574	total: 2m 37s	remaining: 4.41s
4864:	learn: 0.1997574	total: 2m 37s	remaining: 4.38s
4865:	learn: 0.1997574	total: 2m 37s	remaining: 4.34s
4866:	learn: 0.1997574	total: 2m 37s	remaining: 4.31s
4867:	learn: 0.1997574	total: 2m 37s	remaining: 4.28s
4868:	learn: 0.1997574	total: 2m 37s	remaining: 4.25s
4869:	learn: 0.1997574	total: 

In [59]:
sub_cat = np.expm1(cat.predict(X_test))

In [60]:
sub_cat

array([1305.69377049, 1854.17613499, 8269.68378597, ..., 5568.23958641,
       5090.95326728, 4953.00558035])